# Numerical Recipes Workshop 3
For the week of 7-11 October, 2019

This notebook will provide a practical example of root finding for a nonlinear equation.

## The temperature of interstellar dust grains

Understanding the nature of interstellar dust grains is vital to many areas of astronomy, from star formation to measuring the cosmic microwave background (CMB). Many observed properties of interstellar dust are derived from knowing its temperature. In general, dust is well mixed with the gas in the interstellar medium (ISM), but the two are rarely at the same temperature.

The timescales for dust-related processes are very short, so the dust temperature can be calculated by assuming it is in thermal equilibrium at all times. Then, one only needs to balance the various heating and cooling processes, i.e., to find the root of the energy loss equation:

$
\begin{align}
\large
\frac{de}{dt} = \Gamma(T_{dust}) - \Lambda(T_{dust}),
\end{align}
$

where $\Gamma$ and $\Lambda$ are the dust heating and cooling rates, respectively.

Including the relevant heating and cooling processes, this becomes

$
\begin{align}
\large
\frac{de}{dt} = 4 \sigma T_{CMB}^{4} \kappa_{gr} +
\Gamma_{isrf} + \Lambda_{gas/grain}(T_{dust}, T_{gas}, n_{H}) - 4 \sigma T_{dust}^{4} \kappa_{gr},
\end{align}
$

where $\sigma$ is the Stefan-Boltzmann constant, $T_{CMB}$ is the temperature of the CMB, $\kappa_{gr}$ is the dust opacity, $\Gamma_{isrf}$ is the heating from the instellar radiation field, and $\Lambda_{gas/grain}$ is the rate of heat exchange via collisions between the gas and dust. The first term represents heating from the CMB, the second is heating from nearby stars, the third term transfers heat from the hotter to the cooler matter, and the final term is the cooling of the dust by thermal radiation.

The opacity of the dust can be approximated by the piece-wise power-law:

$
\begin{align}
\large
\kappa_{gr}(T_{dust}) \propto \left\{ \begin{array}{ll}
T_{dust}^{2} & , T_{dust} < 200 K,\\
\textrm{constant} & , 200\ K < T_{dust} < 1500\ K,\\
T_{dust}^{-12} & , T_{dust} > 1500\ K.
\end{array} \right.
\end{align}
$

The gas/grain heat transfer rate is given by:

$
\begin{align}
\large
\Lambda_{gas/grain} = 7.2\times10^{-8} n_{H}
\left(\frac{T_{gas}}{1000 K}\right)^{\frac{1}{2}} (1 - 0.8 e^{-75/T_{gas}}) (T_{gas} - T_{dust})\ [erg/s/g],
\end{align}
$

where $n_{H}$ is the number density of the gas.

## Calculating dust temperatures with root finding

The above equations have been coded below with the full heat balance equation implemented as the `gamma_grain` function. Do `help(gamma_grain)` to see how it can be called.

Assuming a constant gas temperature, $T_{gas}$ and gas density, $n_{H}$, calculate the dust temperature, $T_{dust}$, using bisection, the secand method, and the Scipy implementation of [Brent's method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html#scipy.optimize.brentq). Implement your own bisection and secant methods and count the number of steps to reach a solution.

In [7]:
import numpy as np

mh = 1.673735e-24 # g
# Stefan-Boltzmann constant
sigma_b = 5.670373e-5 # erg cm^−2 s^−1 K^−4

def gas_grain(Tgas):
    """
    Return gas/grain heat transfer rate coefficient.
    """

    grain_coef = 1.2e-31 * 1.0e3**-0.5 / mh
    gasgra = grain_coef * Tgas**0.5 * \
        (1.0 - (0.8 * np.exp(-75.0 / Tgas)))
    return gasgra

def kappa_grain(Tdust):
    """
    Return grain mean opacity.
    """

    kgr1   = 4.0e-4
    kgr200 = 16.0
    T_subl = 1500.

    Tdust = np.asarray(Tdust)
    kgr = np.zeros(Tdust.size)

    f1 = Tdust < 200
    if f1.any():
        kgr[f1] = kgr1 * Tdust[f1]**2

    kgr[(Tdust >= 200) & (Tdust < T_subl)] = kgr200

    f2 = Tdust >= T_subl
    if f2.any():
        kgr[f2] = kgr200 * (Tdust[f2] / T_subl)**-12
    
    return kgr

def gamma_isrf():
    """
    Interstellar radiation field heating rate coefficient.
    """

    return 4.154682e-22 / mh

def gamma_grain(Tdust, Tgas, nh, isrf=1.7, z=0):
    """
    Return the grain heating rate.
    
    Parameters
    ----------
    
    Tdust : float
        dust temperature in K
    Tgas : float
        gas temperature in K
    nh : float
        Hydrogen number density in cm^-3
    isrf : float, optional
        interstellar radiation field strengh in Habing units
        default: 1.7 (typical for local interstellar medium)
    z : float, optional
        current redshift, used to set the temperature of the
        Cosmic Microwave Background.
        default: 0
    """

    TCMB = 2.73 * (1 + z)
    my_isrf = isrf * gamma_isrf()

    return my_isrf + \
        4 * sigma_b * kappa_grain(Tdust) * (TCMB**4 - Tdust**4) + \
        (gas_grain(Tgas) * nh * (Tgas - Tdust))

In [8]:
### Tgas and nH values
Tgas = 100 # K
nH = 1e3 # cm^-3

### Bisection
See if you can implement the bisection method to calculate $T_{dust}$ for a relative tolerance of $10^{-4}$, where the relative tolerance is given by:

$
\begin{align}
rtol = \left|\frac{val_{new} - val_{old}}{val_{old}}\right|.
\end{align}
$

A sensible initial bound is $[T_{CMB}, T_{gas}]$, where $T_{CMB} = 2.73 K$ in the local Universe.

In [21]:
def bisection(low, high, tol):
    
    while (np.abs(high - low)) >= tol:
        midpoint = (high + low) / 2.0
        above = gamma_grain(high, Tgas, nH) * gamma_grain(midpoint, Tgas, nH)
        below = gamma_grain(midpoint, Tgas, nH) * gamma_grain(low, Tgas, nH)
        if above < 0:
            low = midpoint
        elif below < 0:
            high = midpoint
        
    return midpoint

answer = bisection(2.73, Tgas, 1e-4)
print(answer)

40.85661255836487


### Secant Method

See if you can implement the secant method for the same tolerance and initial guesses.

In [ ]:
def secant(high, low, tol):
    x_
    while(np.abs(high - low)) >= tol:
        

### Brent's Method

Use [Brent's method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html#scipy.optimize.brentq) to calculate $T_{dust}$. After that, try calculating $T_{dust}$ for a range of $n_{H}$ from $1\ cm^{-3}$ to $10^{13} cm^{-3}$ and plotting $T_{dust}$ vs. $n_{H}$.

In [ ]:
import scipy.optimize as opt

In [ ]:
# Try a range of nH values.
nH = np.logspace(0, 13, 100)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 14

In [ ]:
# plot Tdust vs. nH
plt.semilogx(nH, Tdust)
plt.xlabel('$n_{H}$ $cm^{-3}$')
plt.ylabel('$T_{dust}$ $K$')